# Imports

In [ ]:
from pathlib import Path
from asapdiscovery.data.openeye import load_openeye_sdfs, oechem, save_openeye_sdfs
from asapdiscovery.data.fragalysis import parse_fragalysis
import numpy as np
import pandas as pd, numpy as np
import plotly.express as px
from datetime import datetime
from tqdm.notebook import tqdm
from asapdiscovery.docking.analysis import DockingResults
import asapdiscovery.data.openeye as oe
import asapdiscovery.modeling.cheminformatics as ci
from importlib import reload

# Load Paths

In [ ]:
import sys
sys.path.append(str(Path("../../../").resolve()))
from software.paths import paths

In [ ]:
csv_path = paths.sars_hybrid / "results_cleaned.csv"

In [ ]:
df = pd.read_csv(csv_path)

# load mols

In [ ]:
sdfs = load_openeye_sdfs(str(paths.combined_p_only_sdf))

# Calculate Tanimoto

In [ ]:
reload(ci)

In [ ]:
tc_df = ci.get_n_to_n_tanimoto(sdfs, sdfs)

In [ ]:
self_tc = tc_df[tc_df.Compound_ID == tc_df.Reference_Ligand]

In [ ]:
sum(self_tc.TanimotoCombo == 2.0)

In [ ]:
flipped = pd.DataFrame({"Compound_ID":tc_df.Reference_Ligand, "Reference_Ligand":tc_df.Compound_ID, "TanimotoCombo":tc_df.TanimotoCombo})

In [ ]:
merged = pd.merge(tc_df, flipped, on=["Compound_ID", "Reference_Ligand"])

In [ ]:
px.histogram(merged.TanimotoCombo_x - merged.TanimotoCombo_y)

In [ ]:
px.scatter(flipped, x="Reference_Ligand", y="Compound_ID", color="TanimotoCombo")

# Add to main df

In [ ]:
to_save = pd.merge(df, tc_df, on=["Compound_ID", "Reference_Ligand"])

In [ ]:
to_save = to_save.drop(columns="Unnamed: 0")

In [ ]:
to_save.to_csv(csv_path.parent / "results_cleaned_tc_scores.csv")

In [ ]:
out = csv_path.parent / "results_cleaned_tc_scores.csv"

In [ ]:
out